In [132]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [133]:
patch_sell='../../../sell-in.txt/sell-in.txt'
patch_productos='c:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/tb_productos26052024.txt'
patch_stock='../../../tb_stocks.txt/tb_stocks.txt'
patch_productos_Predecir='C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Datasets/productos_a_predecir.txt'

In [134]:
sell = pd.read_csv(patch_sell, sep='\t')
sell.shape

(2945818, 7)

In [135]:
sell.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [136]:
productos = pd.read_csv(patch_productos, sep='\t')
productos.shape

(1262, 6)

In [137]:
productos.head()

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288


In [138]:
stock = pd.read_csv(patch_stock, sep='\t')
stock.shape

(13691, 3)

In [139]:
stock.head()

,periodo,product_id,stock_final
0,201810,20524,1.61267
1,201810,20311,2.93657
2,201810,20654,6.83269
3,201810,21005,1.01338
4,201810,20974,0.34595


In [140]:
# Unir los DataFrames
sell_productos = pd.merge(sell, productos, on='product_id', how='left')
sell_productos.shape

(2988650, 12)

In [141]:
sell_productos.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0


In [142]:
sell_productos_stock = pd.merge(sell_productos, stock, on=['periodo', 'product_id'], how='left')
sell_productos_stock.shape

(2988650, 13)

In [143]:
sell_productos_stock.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [144]:
productos_a_predecir = pd.read_csv(patch_productos_Predecir, sep='\t')

In [145]:
# Realizar un inner join en los productos a predecir
df_prediccion= pd.merge(sell_productos_stock, productos_a_predecir, on='product_id', how='inner')
df_prediccion.shape

(2332211, 13)

In [146]:
df_prediccion.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [147]:
# Convertir el periodo a formato datetime
df_prediccion['periodo'] = pd.to_datetime(df_prediccion['periodo'], format='%Y%m')
df_prediccion.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,2017-01-01,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,2017-01-01,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,2017-01-01,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,2017-01-01,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,2017-01-01,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [148]:
# Agregar los datos por periodo y product_id para obtener la serie temporal
ts = df_prediccion.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()
ts.head()

,periodo,product_id,tn
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [149]:
ts.shape

(22349, 3)

In [150]:
# Asegurarse de que las columnas tengan el mismo tipo y formato
ts['product_id'] = ts['product_id'].astype(int)
ts['periodo'] = pd.to_datetime(ts['periodo'])

In [151]:
ts.head()

,periodo,product_id,tn
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [152]:
# Agregar lags a tus datos
lags = 3  # Número de lags a incluir
for lag in range(1, lags + 1):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

In [153]:
ts.head()

,periodo,product_id,tn,tn_lag_1,tn_lag_2,tn_lag_3
0,2017-01-01,20001,934.77222,NaN,NaN,NaN
1,2017-01-01,20002,550.15707,NaN,NaN,NaN
2,2017-01-01,20003,1063.45835,NaN,NaN,NaN
3,2017-01-01,20004,555.91614,NaN,NaN,NaN
4,2017-01-01,20005,494.27011,NaN,NaN,NaN


In [154]:
ts.tail()

,periodo,product_id,tn,tn_lag_1,tn_lag_2,tn_lag_3
22344,2019-12-01,21263,0.01270,0.03247,0.01552,0.01128
22345,2019-12-01,21265,0.05007,0.06600,0.10921,0.01707
22346,2019-12-01,21266,0.05121,0.06713,0.11831,0.02844
22347,2019-12-01,21267,0.01569,0.04052,0.09676,0.01830
22348,2019-12-01,21276,0.00892,0.03341,0.02079,0.01856


In [155]:
ts.shape

(22349, 6)

In [156]:
# Eliminar filas con valores nulos resultantes de los lags
ts.dropna(inplace=True)

In [157]:
ts.shape

(20009, 6)

In [158]:
# Obtener la lista de productos únicos a predecir
product_ids = productos_a_predecir['product_id'].unique()

In [159]:
# Crear características adicionales si es necesario (Ejemplo: características temporales)
ts['year'] = ts['periodo'].dt.year
ts['month'] = ts['periodo'].dt.month

In [160]:
ts.head()

,periodo,product_id,tn,tn_lag_1,tn_lag_2,tn_lag_3,year,month
1495,2017-04-01,20001,1069.96130,1303.35771,798.01620,934.77222,2017,4
1496,2017-04-01,20002,522.35365,834.73521,505.88633,550.15707,2017,4
1497,2017-04-01,20003,525.82591,917.16548,752.11520,1063.45835,2017,4
1498,2017-04-01,20004,512.05402,489.91328,508.20044,555.91614,2017,4
1499,2017-04-01,20005,662.59032,563.89955,551.43060,494.27011,2017,4


In [161]:
# Crear conjunto de entrenamiento y objetivo
X = ts[['product_id', 'year', 'month'] + [f'tn_lag_{lag}' for lag in range(1, lags + 1)]]  # Puedes agregar más características si están disponibles
y = ts['tn']

In [162]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [163]:
# Crear y entrenar el modelo LightGBM con Random Forest
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'regression',
    'boosting': 'rf',  # Utilizar Random Forest para generar promedios en el árbol
    'num_iterations': 1000,  # Ajustar la cantidad de iteraciones según sea necesario
    'learning_rate': 0.003934720226268362,
    'num_leaves': 96,
    'bagging_freq': 7,  # Frecuencia de bagging
    'bagging_fraction': 0.6854002560591308,  # Fracción de bagging
    'verbose': -1
}

In [164]:
# Entrenamiento de LightGBM con barra de progreso
evals_result = {}
model = lgb.train(
    params, 
    train_data, 
    valid_sets=[train_data, test_data], 
    valid_names=['train', 'valid']#,
    #evals_result=evals_result,
    #early_stopping_rounds=10, 
    #verbose_eval=10  # Muestra progreso cada 10 iteraciones
)

c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [165]:
# Realizar predicciones para los productos a predecir
results = []
for product_id in tqdm(product_ids, desc="Predicting with LGBM"):
    product_data = ts[ts['product_id'] == product_id].copy()
    if not product_data.empty:
        # Predicción para el último periodo disponible + 1 mes
        last_period = product_data['periodo'].max()
        next_period = last_period + pd.DateOffset(months=1)
        next_data = pd.DataFrame({
            'product_id': [product_id],
            'year': [next_period.year],
            'month': [next_period.month],
            'tn_lag_1': [product_data['tn'].iloc[-1]],
            'tn_lag_2': [product_data['tn_lag_1'].iloc[-1]],
            'tn_lag_3': [product_data['tn_lag_2'].iloc[-1]]
        })
        
        pred = model.predict(next_data)
        results.append({
            'product_id': product_id,
            'predicted_tn': pred[0]
        })

Predicting with LGBM:   0%|          | 0/780 [00:00<?, ?it/s]

Predicting with LGBM: 100%|██████████| 780/780 [00:04<00:00, 193.65it/s]


In [166]:
# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)

# Asegurarse de que el DataFrame resultante tiene las columnas product_id y predicted_tn
results_df = results_df[['product_id', 'predicted_tn']]

# Exportar a un archivo CSV con las columnas product_id y predicted_tn
results_df.to_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/Labo3/datasets/Proyecto/Labo3/Predicciones/lgbm_predictions4.csv', index=False)

print("Predicciones exportadas a 'lgbm_predictions4.csv'")

Predicciones exportadas a 'lgbm_predictions4.csv'
